In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Store CSV into DataFrame

In [2]:
console_csv = "Resources/console.csv"
console_data_df = pd.read_csv(console_csv)
console_data_df.head()

,ConsoleID,Console_Name,Manufacturer,Release_Year,Sales,Type
0,PC,Personal Computer,Computer,1975,1000.00,Home
1,PS2,PlayStation 2,Sony,2000,155.00,Home
2,DS,Nintendo DS,Nintendo,2004,154.02,Handheld
3,GB,Game Boy,Nintendo,1989,118.69,Handheld
4,PS4,PlayStation 4,Sony,2013,108.90,Home


In [3]:
video_games_csv = "Resources/Video_Games.csv"
video_games_data_df = pd.read_csv(video_games_csv)
video_games_data_df.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


# Clean Data

In [4]:
new_console_data_df = console_data_df[['ConsoleID', 'Console_Name', 
                                       'Manufacturer','Release_Year',
                                      'Sales']].copy()
console_data_transformed = new_console_data_df.rename(columns={"ConsoleID":"consoleid",
                                                              "Console_Name":"console_name",
                                                              "Manufacturer":"manufacturer",
                                                              "Release_Year":"release_year",
                                                              "Sales":"sales"})
console_data_transformed.head()

,consoleid,console_name,manufacturer,release_year,sales
0,PC,Personal Computer,Computer,1975,1000.00
1,PS2,PlayStation 2,Sony,2000,155.00
2,DS,Nintendo DS,Nintendo,2004,154.02
3,GB,Game Boy,Nintendo,1989,118.69
4,PS4,PlayStation 4,Sony,2013,108.90


In [5]:
new_video_games_data_df = video_games_data_df[['Name', 'Platform', 
                                       'Year_of_Release','Publisher',
                                      'Global_Sales']].copy()
games_data_transformed = new_video_games_data_df.rename(columns={"Platform": "consoleid", 
                                                                 "Name":"name",
                                                                 "Publisher":"publisher",
                                                                "Year_of_Release":"release_year",
                                                                "Global_Sales":"sales"})
games_data_transformed.head()

,name,consoleid,release_year,publisher,sales
0,Wii Sports,Wii,2006.0,Nintendo,82.53
1,Super Mario Bros.,NES,1985.0,Nintendo,40.24
2,Mario Kart Wii,Wii,2008.0,Nintendo,35.52
3,Wii Sports Resort,Wii,2009.0,Nintendo,32.77
4,Pokemon Red/Pokemon Blue,GB,1996.0,Nintendo,31.37


## Connect to local database

In [6]:
rds_connection_string = "postgres:postgres@localhost:5432/Video_Games_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [7]:
engine.table_names()

['console', 'video_games']

## CSV converted DataFrame into database

In [8]:
console_data_transformed.to_sql(name='console', con=engine, if_exists='append', index=False)

In [9]:
games_data_transformed.to_sql(name='video_games', con=engine, if_exists='append', index=False)

## Load DataFrames into database

In [10]:
pd.read_sql_query('select * from console', con=engine).head()

,consoleid,console_name,manufacturer,release_year,sales
0,PC,Personal Computer,Computer,1975,1000
1,PS2,PlayStation 2,Sony,2000,155
2,DS,Nintendo DS,Nintendo,2004,154
3,GB,Game Boy,Nintendo,1989,119
4,PS4,PlayStation 4,Sony,2013,109


In [11]:
pd.read_sql_query('select * from video_games', con=engine).head()

,name,consoleid,release_year,publisher,sales
0,Wii Sports,Wii,2006.0,Nintendo,83
1,Super Mario Bros.,NES,1985.0,Nintendo,40
2,Mario Kart Wii,Wii,2008.0,Nintendo,36
3,Wii Sports Resort,Wii,2009.0,Nintendo,33
4,Pokemon Red/Pokemon Blue,GB,1996.0,Nintendo,31
